In [1]:
import pandas as pd
import geopandas as gpd
import datetime as dt
import numpy as np
import sqlite3

## Import The Wildfire Dataset
- Import all wildfires from California from the SQLite database, download [here](https://www.kaggle.com/rtatman/188-million-us-wildfires)
- Check for missing data
- Identify data columns worth keeping

In [2]:
con = sqlite3.connect("FPA_FOD_20170508.sqlite")
fires = pd.read_sql_query('SELECT * FROM Fires WHERE STATE == "CA"', con)
fires.head(5)

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,...,A,40.036944,-121.005833,5.0,USFS,CA,63,063,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.933056,-120.404444,5.0,USFS,CA,61,061,Placer,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\...
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.984167,-120.735556,13.0,STATE OR PRIVATE,CA,17,017,El Dorado,b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\...
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.913333,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]...
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.933056,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\...


## Inspect The Data
- Review aspects of the dataset
- Identify unecessary columns
- Check for missing data

In [3]:
print(fires.shape)
print(fires.columns)
print(fires.isnull().sum())

(189550, 39)
Index(['OBJECTID', 'FOD_ID', 'FPA_ID', 'SOURCE_SYSTEM_TYPE', 'SOURCE_SYSTEM',
       'NWCG_REPORTING_AGENCY', 'NWCG_REPORTING_UNIT_ID',
       'NWCG_REPORTING_UNIT_NAME', 'SOURCE_REPORTING_UNIT',
       'SOURCE_REPORTING_UNIT_NAME', 'LOCAL_FIRE_REPORT_ID',
       'LOCAL_INCIDENT_ID', 'FIRE_CODE', 'FIRE_NAME',
       'ICS_209_INCIDENT_NUMBER', 'ICS_209_NAME', 'MTBS_ID', 'MTBS_FIRE_NAME',
       'COMPLEX_NAME', 'FIRE_YEAR', 'DISCOVERY_DATE', 'DISCOVERY_DOY',
       'DISCOVERY_TIME', 'STAT_CAUSE_CODE', 'STAT_CAUSE_DESCR', 'CONT_DATE',
       'CONT_DOY', 'CONT_TIME', 'FIRE_SIZE', 'FIRE_SIZE_CLASS', 'LATITUDE',
       'LONGITUDE', 'OWNER_CODE', 'OWNER_DESCR', 'STATE', 'COUNTY',
       'FIPS_CODE', 'FIPS_NAME', 'Shape'],
      dtype='object')
OBJECTID                           0
FOD_ID                             0
FPA_ID                             0
SOURCE_SYSTEM_TYPE                 0
SOURCE_SYSTEM                      0
NWCG_REPORTING_AGENCY              0
NWCG_REPORTING_UNI

## Clean The Data
- Drop columns that are mostly empty and ones we don't need for analysis
- Create datetime and month column for future analysis

In [4]:
cols = ['LOCAL_FIRE_REPORT_ID', 'LOCAL_INCIDENT_ID', 'FIRE_CODE',
        'FIRE_NAME', 'ICS_209_INCIDENT_NUMBER', 'ICS_209_NAME',
        'MTBS_ID', 'MTBS_FIRE_NAME', 'COMPLEX_NAME', 'DISCOVERY_TIME', 'STAT_CAUSE_CODE',
        'CONT_DATE', 'CONT_DOY', 'CONT_TIME', 'FOD_ID', 'FPA_ID', 'SOURCE_SYSTEM_TYPE',
        'SOURCE_SYSTEM', 'NWCG_REPORTING_AGENCY', 'NWCG_REPORTING_UNIT_ID',
        'NWCG_REPORTING_UNIT_NAME', 'SOURCE_REPORTING_UNIT', 'SOURCE_REPORTING_UNIT_NAME',
        'OWNER_CODE', 'OWNER_DESCR', 'Shape', 'STATE', 'COUNTY',
        'FIPS_CODE', 'FIPS_NAME'
        ]
        
fires = fires.drop(columns=cols, axis=1)
fires.head(5)

,OBJECTID,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE
0,1,2005,2453403.5,33,Miscellaneous,0.10,A,40.036944,-121.005833
1,2,2004,2453137.5,133,Lightning,0.25,A,38.933056,-120.404444
2,3,2004,2453156.5,152,Debris Burning,0.10,A,38.984167,-120.735556
3,4,2004,2453184.5,180,Lightning,0.10,A,38.559167,-119.913333
4,5,2004,2453184.5,180,Lightning,0.10,A,38.559167,-119.933056


- Create DATETIME column in YYYY-MM-DD format

In [5]:
dts = []
for index, row in fires.iterrows():
  dts.append(dt.datetime.strptime(str(row['FIRE_YEAR']) + "-" + str(row['DISCOVERY_DOY']), "%Y-%j").strftime("%m-%d-%Y"))

fires['DATETIME'] = dts
fires.DATETIME = pd.to_datetime(fires.DATETIME)
fires = fires.drop(columns=['DISCOVERY_DATE', 'DISCOVERY_DOY'], axis=1)
fires['MONTH'] = fires['DATETIME'].dt.month_name()
fires.head(5)

,OBJECTID,FIRE_YEAR,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,DATETIME,MONTH
0,1,2005,Miscellaneous,0.10,A,40.036944,-121.005833,2005-02-02,February
1,2,2004,Lightning,0.25,A,38.933056,-120.404444,2004-05-12,May
2,3,2004,Debris Burning,0.10,A,38.984167,-120.735556,2004-05-31,May
3,4,2004,Lightning,0.10,A,38.559167,-119.913333,2004-06-28,June
4,5,2004,Lightning,0.10,A,38.559167,-119.933056,2004-06-28,June


## Assign FIPS Codes
- The fires dataset is missing a lot of FIPS code values
- Use US Census block groups to assign the missing values, download [here](http://www2.census.gov/geo/tiger/TIGER2020/BG/tl_2020_06_bg.zip)

- Import the US Census block groups for California
- Create County FIPS column to be assigned to the fires dataset

In [6]:
block_groups = gpd.read_file("tl_2020_06_bg/tl_2020_06_bg.shp").to_crs(epsg=4326)
block_groups['STCT_FIPS'] = block_groups['STATEFP']+block_groups['COUNTYFP']
block_groups.head(5)

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,STCT_FIPS
0,06,053,001500,1,060530015001,Block Group 1,G5030,S,3348560,0,+36.6564570,-121.6749086,"POLYGON ((-121.69184 36.65885, -121.68510 36.6...",06053
1,06,073,002712,2,060730027122,Block Group 2,G5030,S,337744,0,+32.7303521,-117.0787620,"POLYGON ((-117.08306 32.72588, -117.08280 32.7...",06073
2,06,073,018601,2,060730186012,Block Group 2,G5030,S,886725,0,+33.2295683,-117.3482855,"POLYGON ((-117.35296 33.22489, -117.35294 33.2...",06073
3,06,073,018609,3,060730186093,Block Group 3,G5030,S,379311,0,+33.2495250,-117.3148675,"POLYGON ((-117.31819 33.25084, -117.31819 33.2...",06073
4,06,073,019406,3,060730194063,Block Group 3,G5030,S,649440,0,+33.2018708,-117.2686502,"POLYGON ((-117.27370 33.19762, -117.27367 33.1...",06073


- Create point geometry from the fires dataset
- Join the block groups values to the fires based on fire point location

In [7]:
fires_points = gpd.GeoDataFrame(fires, geometry=gpd.points_from_xy(fires.LONGITUDE, fires.LATITUDE))
fires_points.crs = {'init': 'epsg:4326', 'no_defs': True}

block_groups = block_groups[['GEOID','STCT_FIPS','geometry']]
fires_block = gpd.sjoin(fires_points, block_groups, how="inner", op='intersects')

In [8]:
fires_block.head(5)

,OBJECTID,FIRE_YEAR,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,DATETIME,MONTH,geometry,index_right,GEOID,STCT_FIPS
0,1,2005,Miscellaneous,0.10,A,40.036944,-121.005833,2005-02-02,February,POINT (-121.00583 40.03694),16901,060630004002,06063
169,1446,2005,Miscellaneous,0.10,A,40.004722,-121.260556,2005-08-24,August,POINT (-121.26056 40.00472),16901,060630004002,06063
184,1793,2005,Miscellaneous,0.10,A,40.093056,-120.910000,2005-08-25,August,POINT (-120.91000 40.09306),16901,060630004002,06063
1058,8127,2006,Lightning,0.20,A,40.035833,-121.020278,2006-06-14,June,POINT (-121.02028 40.03583),16901,060630004002,06063
1099,8356,2006,Lightning,0.25,A,40.104167,-120.861667,2006-06-26,June,POINT (-120.86167 40.10417),16901,060630004002,06063


## Export the Final Dataset
- Decide the final format
- Confirm there is no missing data
- export to csv

In [9]:
cols = ['OBJECTID','FIRE_YEAR','STAT_CAUSE_DESCR','FIRE_SIZE','FIRE_SIZE_CLASS','LATITUDE','LONGITUDE','GEOID','STCT_FIPS','DATETIME','MONTH']
final_fires_cleaned = fires_block[cols]
final_fires_cleaned.head(5)

,OBJECTID,FIRE_YEAR,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,GEOID,STCT_FIPS,DATETIME,MONTH
0,1,2005,Miscellaneous,0.10,A,40.036944,-121.005833,060630004002,06063,2005-02-02,February
169,1446,2005,Miscellaneous,0.10,A,40.004722,-121.260556,060630004002,06063,2005-08-24,August
184,1793,2005,Miscellaneous,0.10,A,40.093056,-120.910000,060630004002,06063,2005-08-25,August
1058,8127,2006,Lightning,0.20,A,40.035833,-121.020278,060630004002,06063,2006-06-14,June
1099,8356,2006,Lightning,0.25,A,40.104167,-120.861667,060630004002,06063,2006-06-26,June


In [10]:
final_fires_cleaned.isnull().sum()

OBJECTID            0
FIRE_YEAR           0
STAT_CAUSE_DESCR    0
FIRE_SIZE           0
FIRE_SIZE_CLASS     0
LATITUDE            0
LONGITUDE           0
GEOID               0
STCT_FIPS           0
DATETIME            0
MONTH               0
dtype: int64

In [11]:
final_fires_cleaned.to_csv('final_fires_cleaned.csv')